<div style="background-color: #ffffff ; padding: 10px;">

**Goals**:
1. Create a new CSV file that lists: (1) Country Name; (2) latitude and longitude; (3) annual inflation rate for the last twenty years.
2. Delete regions listed in inflation data.
3. Delete countries that do not contain any inflation data for the last 20 years.

**Data**:
* ["world_inflation_data.csv"](world_inflation_data.csv) (World Bank, Creative Commons)
* ["world_geo_data.csv"](world_geo_data.csv) (Paul Mooney - Kaggle, Creative Commons)
* ["country_codes_data.csv"](country_codes_data.csv) (Luke Dunalfe – Github, Creative Commons)

**Skills**: Pandas DataFrame Configuration, Data Cleaning, Data Masking, Indexing and Sorting

**Technology**: Python, Pandas

**Results**: ["country_inflation_geo_data.csv"](./country_inflation_geo_data.csv)
</div>

In [1]:
# imports
from pathlib import Path
import pandas as pd

In [2]:
# set pandas to display all rows by default for manual review
pd.set_option("display.max_rows", None)

In [3]:
# create dataframes from CSV files
world_geo_df = pd.read_csv("world_geo_data.csv")
world_inflation_df = pd.read_csv("world_inflation_data.csv", header=4)
world_country_codes_df = pd.read_csv("country_codes_data.csv")

# preview data head sample from df
print(f"world_geo_df")
display(world_geo_df.head(3))

print(f"world_inflation_df")
display(world_inflation_df.head(3))

print(f"world_country_codes_df")
display(world_country_codes_df.head(3))

world_geo_df


,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas


world_inflation_df


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.750981,5.370290,5.245878,6.571396,6.399343,4.720811,4.653665,7.321106,6.824727,10.773751
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN


world_country_codes_df


,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN


In [4]:
# filter world_inflation_df for desired data (country, country code, inflation data)
world_inflation_filtered_df = pd.concat([world_inflation_df.iloc[:,:2], world_inflation_df.iloc[:,-20:]], axis=1)
    
# merge dfs, based on world_country_codes_df to eliminate non-countries in inflation data
joined_df = pd.merge(world_country_codes_df[["alpha-2", "alpha-3"]], world_inflation_filtered_df,left_on="alpha-3", right_on="Country Code", how="inner")
joined_df = pd.merge(world_geo_df[["country_code", "latitude", "longitude"]], joined_df, right_on="alpha-2", left_on="country_code", how="inner")

# mask inflation data - creates boolean series
mask = joined_df.iloc[:,2:].notna().all(axis=1)

# filter df with mask - based on index in boolean series
joined_df = joined_df[mask]

# drop country codes
joined_df.drop(columns=["country_code", "Country Code", "alpha-2", "alpha-3"], inplace=True)

# set index
joined_df.set_index("Country Name", inplace=True)

# sort by country name
joined_df.sort_index(ascending=True, inplace=True)

# look at entire df for manual review
joined_df


,latitude,longitude,2003,2004,2005,2006,2007,2008,2009,2010,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country Name,,,,,,,,,,,,,,,,,,,,,
Albania,41.153332,20.168331,0.484003,2.280019,2.366582,2.370728,2.932682,3.320871,2.266922e+00,3.626047,...,1.937621,1.625865,1.896174,1.275432,1.986661,2.028060,1.411091,1.620887,2.041472,6.725203
Algeria,28.033886,1.659626,4.268954,3.961800,1.382447,2.311499,3.678996,4.858591,5.737060e+00,3.911062,...,3.254239,2.916927,4.784447,6.397695,5.591116,4.269990,1.951768,2.415131,7.226063,9.265516
Antigua and Barbuda,17.060816,-61.796428,1.993531,2.030078,2.098752,1.787785,1.416053,5.333806,-5.501600e-01,3.370025,...,1.059498,1.089442,0.968993,-0.489438,2.432488,1.207158,1.431356,0.625990,2.062996,7.531078
Armenia,40.069099,45.038189,4.721553,6.961261,0.638928,2.892357,4.407361,8.949953,3.406767e+00,8.176361,...,5.789668,2.981309,3.731691,-1.403608,0.969553,2.520234,1.443447,1.211436,7.184836,8.640911
Australia,-25.274398,133.775136,2.732596,2.343255,2.691832,3.555288,2.327611,4.350299,1.771117e+00,2.918340,...,2.449889,2.487923,1.508367,1.276991,1.948647,1.911401,1.610768,0.846906,2.863910,6.594097
Austria,47.516231,14.550072,1.355554,2.061206,2.299138,1.441549,2.168555,3.215950,5.063088e-01,1.813534,...,2.000156,1.605812,0.896563,0.891592,2.081269,1.998380,1.530896,1.381911,2.766667,8.546870
Azerbaijan,40.143105,47.576927,2.233865,6.708930,9.679507,8.328925,16.699755,20.849087,1.457048e+00,5.726872,...,2.415717,1.373442,4.027686,12.443375,12.935918,2.268547,2.610572,2.759809,6.650299,13.852259
"Bahamas, The",25.034280,-77.396280,3.025045,0.981914,1.591606,2.389826,2.492578,4.489560,2.062735e+00,1.344027,...,0.722415,1.513756,1.861483,-0.346377,1.518207,2.265863,2.491235,0.038521,2.904913,5.605406
Bahrain,25.930414,50.637772,1.593625,2.352941,2.586207,2.007470,3.256667,3.526003,2.795513e+00,1.961885,...,3.300012,2.647553,1.848627,2.786793,1.386718,2.087669,1.005820,-2.317706,-0.606319,3.625741


In [5]:
# save as csv
joined_df.to_csv("country_inflation_geo_data.csv")